In [12]:

#--------------------------------------------------------------------------------------------
#---------------------------------Part 1: DB connection--------------------------------------
#--------------------------------------------------------------------------------------------

import numpy as np
import pandas as pd
import os.path
from scipy.stats import iqr
import numpy as np
import datetime
import time

import mysql.connector
import warnings
warnings.filterwarnings("ignore")



# Initiate with Parameters
db_name= "core_stats"
col = "peak_upload_speed"
export_full_option=False

# Start Database Connection
db_connection = mysql.connector.connect(
    host="10.1.2.10",
    user="gyan",
    password="5Gaa$2022",
    database="gyan_db"
)

# Create Database Cursor for SQL Queries
mycursor = db_connection.cursor(dictionary=True)
mycursor.execute("select * from core_stats WHERE client_id='BETBEL01GYN001' AND stats_timestamp>'2022-07-15' ")
json_data=mycursor.fetchall() ## The fetchall() is one of Python’s cursor methods used to retrieve all the rows for a certain query.

df = pd.DataFrame(json_data)


def add_Z_score_column(df,col, z_score_threshold=3):
    col_name =  "Z-score_" + col
    Z_score_list = (df[col] - df[col].mean())/df[col].std(ddof=0)
    Z_score_list= [np.round(x,2) for x in Z_score_list]
    df[col_name]=Z_score_list
    df["label_Z-score_"+col] = [ 1 if (x< -1*z_score_threshold or x>1*z_score_threshold) else 0 for x in df[col_name]]
    
    deviation_list = []
    value_list = df[col]
    mean_value= df[col].mean()

    for item in value_list:
        if mean_value == 0:
            deviation_list.append(0)
        else:
            deviation_list.append(np.round((item-mean_value) / mean_value ,2)) 

    df["deviation_Z-score_"+col]=  deviation_list
    
    return df



def add_outlier_column(df, col, iqr_factor=2, low_quantile=25, upper_quantile=75):
    q3, q1 = np.percentile(df[col], [upper_quantile, low_quantile])

    iqr_v = iqr(df[col])

    upper_v = q3+iqr_factor*iqr_v
    lower_v = q1-iqr_factor*iqr_v
    
    #outliers_removed = [x for x in df[col] if x >= lower_v and x <= upper_v]
    #print(outliers_removed)
    
    outlier_index = df[(df[col] > upper_v) | (df[col] < lower_v)][col].index
    outlier_label = [1 if x in outlier_index else 0 for x in df.index]
    
    df["label_outlier_"+col]  = outlier_label
    
    return df


def reorder_columns(dataframe, col_name, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string): column name to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    temp_col = dataframe[col_name]
    dataframe = dataframe.drop(columns=[col_name])
    dataframe.insert(loc=position, column=col_name, value=temp_col)
    return dataframe



def insert_anomaly_to_database(current_anomlies):
    for index, row in current_anomlies.iterrows():
        connection = mysql.connector.connect(
            host="10.1.2.10",
            user="gyan",
            password="5Gaa$2022",
            database="gyan_db"
        )

        cursor = connection.cursor()

        MySQL_insert_query = "INSERT INTO tb_export_anomaly_df (client_id, stats_timestamp, attribute_name, attribute_value, attribute_label_Z_score, attribute_deviation, attribute_label_outlier,attribute_mean) VALUES (%s, %s, %s, %s, %s, %s, %s,%s)"

        the_value= (row.client_id, str(row.stats_timestamp),row.Attribute_Name, row.Attribute_Value, row.Attribute_Label_Z_Score, row.Attribute_Deviation_Z, row.Attribute_Label_Outlier, row.attribute_mean)
        
        try:
            cursor.execute(MySQL_insert_query, the_value)
        except:
            print("Record Already Inserted")
            pass

        connection.commit()

    cursor.close()
    print("Insert Complete")


#---------------------------------------------------------------------------------------------
#---------------------------------Part 3: Data Cleaning --------------------------------------
#---------------------------------------------------------------------------------------------


keeped_column_name =list(df.columns)
keeped_column_name.remove("client_id")
keeped_column_name.remove("total_tx_data")
keeped_column_name.remove("total_rx_data")
keeped_column_name.remove("stats_timestamp")
#["client_id","total_tx_data","total_rx_data"]

# Create a new table to store all the computed metrics
Anomaly_table = df.copy()
client_id_list= list(df.client_id.unique())


## Statistics Anomalies and Outlier Anomalies
for clientID in client_id_list:
    temp_df = Anomaly_table[Anomaly_table["client_id"]==clientID]
    
    for col in keeped_column_name:

        temp_df=add_Z_score_column(temp_df,col)
        temp_df=add_outlier_column(temp_df,col)
    

    Anomaly_table=Anomaly_table.append(temp_df, ignore_index = True)

filter_col = [col for col in Anomaly_table if col.startswith('label')]





#------------------------------------------------------------------------------------------
#---------------------------------Part 4: Prepare Export Anomaly Dataframe-----------------
#------------------------------------------------------------------------------------------

# folder_path = "C:\Users\Jijun Du\Desktop\Main_Anomaly_Detection\Generated_csv\"
# Filter columns with at least one record that have anomaly label
export_anomaly_df= pd.DataFrame()
for col in keeped_column_name:
    condition= (Anomaly_table["label_Z-score_"+ col]==1) | (Anomaly_table["label_outlier_"+col]==1)


    subset_columns=["client_id","stats_timestamp",col,"label_Z-score_"+col, "deviation_Z-score_"+col,"label_outlier_"+col]
    #print("{} rows of anomaly detected for column {}".format(sum(condition),col))
    
    subset_Summary= Anomaly_table[condition][subset_columns]
    
    subset_Summary["Attribute_Name"] = col
    
    subset_Summary = reorder_columns(subset_Summary,"Attribute_Name",2)
    #subset_Summary.drop(col, axis=1)
#print(subset_Summary.shape[1])
#print(subset_Summary.columns)
    subset_Summary=subset_Summary.rename(columns={str(col):"Attribute_Value","label_Z-score_"+col: "Attribute_Label_Z_Score",  "deviation_Z-score_"+col: "Attribute_Deviation_Z","label_outlier_"+col: "Attribute_Label_Outlier"})
    
    export_anomaly_df=export_anomaly_df.append(subset_Summary, ignore_index = True)


    
mean_list= []
for i in range(export_anomaly_df.shape[0]):
    try:

        attribute_condition = export_anomaly_df.iloc[i].Attribute_Name
        client_condition = mean_summary["client_id"] == export_anomaly_df.iloc[i].client_id
        value= mean_summary[client_condition][attribute_condition].values[0]
        mean_list.append(value)
    except:
        mean_list.append(0)
        
export_anomaly_df["attribute_mean"]= mean_list


#------------------------------------------------------------------------------------------
#---------------------------------Part 5: Get Current Anomalies and Insert-----------------
#------------------------------------------------------------------------------------------

time_interval = datetime.datetime.now() - datetime.timedelta(minutes=120)
current_anomlies= export_anomaly_df[export_anomaly_df.stats_timestamp > time_interval].reset_index()


if current_anomlies.shape[0]>0:
    insert_anomaly_to_database(current_anomlies)
    print("Anomalies Insert Completed")
else:
    print("No Anomalies Detected")

Insert Complete
Anomalies Insert Completed


In [7]:
current_anomlies.Attribute_Deviation_Z

0     1.33
1     1.30
2     1.27
3     1.64
4     2.23
5     2.23
6     2.23
7     0.34
8     0.73
9     0.64
10    0.46
11    0.69
12    0.92
13    0.69
14    0.87
15    2.22
16    2.02
17    2.22
18    2.22
19    2.04
20    2.03
21    2.03
22    1.52
23    1.32
24    1.45
25    2.24
26    2.24
27    2.03
Name: Attribute_Deviation_Z, dtype: float64

In [3]:
time_interval = datetime.datetime.now() - datetime.timedelta(minutes=30)
current_anomlies= export_anomaly_df[export_anomaly_df.stats_timestamp > time_interval].reset_index()
current_anomlies

,index,client_id,stats_timestamp,Attribute_Name,Attribute_Value,Attribute_Label_Z_Score,Attribute_Deviation_Z,Attribute_Label_Outlier,attribute_mean
0,1,BETBEL01GYN001,2022-09-06 14:32:22,total_attached_user,69.0,1.0,1.33,0.0,0
1,2,BETBEL01GYN001,2022-09-06 14:34:22,total_attached_user,68.0,1.0,1.30,0.0,0
2,3,BETBEL01GYN001,2022-09-06 14:40:22,total_attached_user,67.0,1.0,1.27,0.0,0
3,4,BETBEL01GYN001,2022-09-06 14:42:22,total_attached_user,78.0,1.0,1.64,0.0,0
4,6,BETBEL01GYN001,2022-09-06 14:32:22,total_rejected_user,8.0,1.0,2.23,0.0,0
5,7,BETBEL01GYN001,2022-09-06 14:34:22,total_rejected_user,8.0,1.0,2.23,0.0,0
6,8,BETBEL01GYN001,2022-09-06 14:40:22,total_rejected_user,8.0,1.0,2.23,0.0,0
7,246,BETBEL01GYN001,2022-09-06 14:32:22,peak_upload_speed,60213.0,1.0,0.34,1.0,0
8,247,BETBEL01GYN001,2022-09-06 14:34:22,peak_upload_speed,77487.0,1.0,0.73,1.0,0
9,248,BETBEL01GYN001,2022-09-06 14:36:22,peak_upload_speed,73699.0,1.0,0.64,1.0,0


In [4]:

if current_anomlies.shape[0]>0:
    insert_anomaly_to_database(current_anomlies)
    print("Anomalies Insert Completed")
else:
    print("No Anomalies Detected")

Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Record Already Inserted
Insert Complete
Anomalies Insert Completed
